# Takeda - List
* 2018 OCR, now readable

In [2]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib
import re

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *
import consts

## Import PDF

In [3]:
# Read pdf into DataFrame
df = tabula.read_pdf("switzerland-efpia-transparency-disclosure-tov-2019_tpch.pdf", pages='all', lattice=True)

## Format Table

In [19]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'empty1', 'total', 'empty2']

#Shift
df_export[df_export['name'].isna()] = df_export.shift(-1, axis='columns')

#Remove empty
df_export.drop(columns=['empty1', 'empty2'], inplace=True)

#Remove rows which have no values
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export.reset_index(drop=True)

#Type by text
df_export = set_type_by_alphabetical_order(df_export)

#Revert name
df_export = revert_name(df_export, ' ')

#Extract PLZ
df_export['plz'] = df_export.location.str.extract(r'(\d{4})')
df_export['location'] = df_export.location.str.replace(r'(\d{4}) ', '')

#Cleanup Numbers
df_export = remove_comma(df_export)
df_export = remove_in_numbers(df_export, ' CHF')
df_export = amounts_to_number(df_export)

#Remove \r
df_export = remove_carination(df_export, " ")

#basic string conversion
df_export = basic_string_conversion(df_export)

#Save
export_list(df_export, 'takeda')

Total nicht Summe der Werte
saved


In [46]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx', open=True)